In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quadrature
from scipy.optimize import least_squares 
from datetime import datetime as dt
from eod import EodHistoricalData
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols
import py_vollib_vectorized

In [ ]:
S0=15922

In [ ]:
df1= pd.read_csv('/content/dyh23-volatility-greeks-exp-03_18_23-show-all-02-04-2023.csv')
df2= pd.read_csv('/content/dym23-volatility-greeks-exp-04_22_23-%moneyness%-02-04-2023.csv')
df3= pd.read_csv('/content/dym23-volatility-greeks-exp-06_17_23-%moneyness%-02-04-2023.csv')
df4= pd.read_csv('/content/dyu23-volatility-greeks-exp-09_16_23-%moneyness%-02-04-2023.csv')
df5= pd.read_csv('/content/dyz23-volatility-greeks-exp-12_16_23-%moneyness%-02-04-2023.csv')

In [ ]:
df1['exp']=dt(2023, 3, 18)
df2['exp']=dt(2023, 4, 22)
df3['exp']=dt(2023, 6, 17)
df4['exp']=dt(2023, 9, 23)
df5['exp']=dt(2023, 12, 23)

In [ ]:
df_call = pd.concat([df1[df1['Type']=='Call'], df2[df2['Type']=='Call'], df3[df3['Type']=='Call'], df4[df4['Type']=='Call'], df5[df5['Type']=='Call']], ignore_index=True)
df_put = pd.concat([df1[df1['Type']=='Put'], df2[df2['Type']=='Put'], df3[df3['Type']=='Put'], df4[df4['Type']=='Put'], df5[df5['Type']=='Put']], ignore_index=True)

In [ ]:
df_call['Time']=dt(2023, 2, 4)
df_put['Time']=dt(2023, 2, 4)

In [ ]:
df_call['tau']=(df_call['exp']-df_call['Time']).dt.days/365
df_put['tau']=(df_put['exp']-df_put['Time']).dt.days/365

In [ ]:
df_call['IV']=df_call['IV'].str.strip('%').to_numpy('float')
df_put['IV']=df_put['IV'].str.strip('%').to_numpy('float')

In [ ]:
df_call.head()

,Strike,Type,Last,IV,Delta,Gamma,Theta,Vega,IV Skew,Time,exp,Last Trade,tau
0,4500,Call,10979.5,172.50,0.994668,4.243346e-115,-4.015857e-112,1.896728e-110,+158.26%,2023-02-04,2023-03-18,NaN,0.115068
1,5000,Call,10480.9,157.50,0.994668,5.154146e-97,-5.884828e-94,2.303845e-92,+143.26%,2023-02-04,2023-03-18,NaN,0.115068
2,5500,Call,9982.2,143.94,0.994668,4.872158e-82,-6.592205e-79,2.177800e-77,+129.69%,2023-02-04,2023-03-18,NaN,0.115068
3,6000,Call,9483.6,131.69,0.994668,1.627567e-69,-2.571346e-66,7.275042e-65,+117.45%,2023-02-04,2023-03-18,NaN,0.115068
4,6500,Call,8984.9,120.41,0.994668,5.819387e-59,-1.060279e-55,2.601200e-54,+106.16%,2023-02-04,2023-03-18,NaN,0.115068


In [ ]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    
    # constants
    a = kappa*theta
    b = kappa+lambd
    
    # common terms w.r.t phi
    rspi = rho*sigma*phi*1j
    
    # define d parameter given phi and b
    d = np.sqrt( (rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2 )
    
    # define g parameter given phi, b and d
    g = (b-rspi+d)/(b-rspi-d)
    
    # calculate characteristic function by components
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)
    return exp1*term2*exp2

In [ ]:
(10-1j)*1j

(1+10j)

In [ ]:
#v0, kappa, theta, sigma, rho, lambd
def integrand(phi, S0,  v0, kappa, theta, sigma, rho, lambd, tau, r, K):

  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  
  
  numerator = np.exp(r*tau) * heston_charfunc(phi-1j,*args) - K*heston_charfunc(phi,*args)
  
  denominator = 1j*phi*K**(1j*phi)
  return numerator/denominator

In [ ]:
def vectorized_quad(integrand, S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    result = np.zeros(len(K))
    for i in range(len(K)):
        t = tau[i]
        result[i], _ = quadrature(integrand, 0, 100, maxiter = 25, args=(S0, v0, kappa, theta, sigma, rho, lambd, t, r[i], K[i]))
    return result

In [ ]:
def heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):

  
  
  real_integral = np.real(vectorized_quad(integrand, S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r) )
    
  return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi

Where I found the rates for the Germany Governments bonds
http://www.worldgovernmentbonds.com/country/germany/

In [ ]:
rates = {1/12:	2.440,	
3/12:	2.5782,	
6/12:	2.725,	
9/12:	2.940,	
1:	2.919,	
2:	2.765,	
3:  2.516,	
4:	2.409,	
5:	2.399,	
6:	2.319,	
7:	2.303,	
8:	2.303,	
9:	2.313,	
10:	2.362,	
15:	2.439,	
20:	2.410,	
25:	2.328,	
30:	2.320}	

In [ ]:
mat = []
rate = []
for i, j in rates.items():
  mat.append(i)
  rate.append(j/100)
mat = np.array(mat)
rate = np.array(rate)

In [ ]:
curve_fit, status = calibrate_nss_ols(mat, rate) 
curve_fit

NelsonSiegelSvenssonCurve(beta0=0.027281114404616363, beta1=-0.0008170676412227167, beta2=0.014071560272008411, beta3=-0.023836160555950073, tau1=2.0, tau2=5.0)

In [ ]:
df_call['r']= df_call['tau'].apply(curve_fit)
df_put['r']= df_put['tau'].apply(curve_fit)
K = df_call['Strike'].to_numpy('float') # strike
tau = df_call['tau']#df_call['tau'] # time to maturity
r = df_call['r']

In [ ]:
iv=df_call['IV']/100

In [ ]:
def heston_ivol(x):
  v0, kappa, theta, sigma, rho, lambd = [param for param in x]
  P = heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
  impl = py_vollib_vectorized.vectorized_implied_volatility(P, S0, K, tau, r, flag = ['c'], q=0, model='black_scholes',return_as='numpy', on_error='ignore')
  diff = iv - impl
  return diff


In [ ]:
def calibrate_heston(df, S0, x, r):
    
  K=df['Strike']
  tau=df['tau']
  iv = df['IV']/100
  r=df['r']
  S0=S0

  #market_ivol = heston_ivol(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r, iv)
  res = least_squares(heston_ivol, x0=np.array(x), verbose=0, max_nfev=100, method='lm')
  return res.x

In [ ]:
#v0, kappa, theta, sigma, rho, lambd
v0_dict = {'a': 0.1838, 'b': 0.0400, 'c':0.1500, 'd': 0.0231, 'e':0.0654}
kappa_dict = {'a': 6.5482, 'b': 1.1500, 'c': 3.0000, 'd': 1.3784, 'e': 0.6067}
theta_dict = {'a': 0.0731, 'b': 0.0400, 'c': 0.0500, 'd': 0.2319, 'e': 0.0707}
sigma_dict = {'a': 2.3012, 'b': 0.2000, 'c': 0.5000, 'd':1.0359, 'e': 0.2928}
rho_dict = {'a': -0.4176, 'b': -0.4000, 'c': -0.5000, 'd': -0.2051, 'e': -0.7571}
lambd_dict = {'a':0, 'b':0, 'c':0, 'd':0, 'e':0}


In [ ]:
params={}
for i in v0_dict.keys():
  
  params[i]=calibrate_heston(df_call, S0, [v0_dict[i], kappa_dict[i], theta_dict[i], sigma_dict[i], rho_dict[i], lambd_dict[i]], r)

In [ ]:
params['a']

array([1.81399184e-01, 1.14899794e+01, 5.55634143e-03, 1.45734313e-01,
       2.56457237e+00, 4.94233382e+00])

In [ ]:
params['b']

array([ 0.09494881,  1.49211037, -0.0603517 , -0.16333182, -0.91476436,
        0.79519401])

In [ ]:
params['c']

array([ 2.36907886e-01,  4.44876140e+00,  1.03543590e-05, -1.50083126e-01,
       -2.96345669e+00,  1.84431438e+01])

In [ ]:
params['d']

array([ 0.10013477,  1.92964872, -0.0381771 , -0.25970608, -0.96594959,
        0.69050026])

In [ ]:
params['e']

array([ 0.08310475,  0.62106296, -0.1412369 ,  0.01705458, -0.67061454,
        0.0242637 ])

In [ ]:
err = {}
for i in params.keys():
  err[i]=(heston_ivol(params[i].tolist())**2).sum()/K.shape[0]

<ipython-input-48-7b73f4590402>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  result[i], _ = quadrature(integrand, 0, 100, maxiter = 25, args=(S0, v0, kappa, theta, sigma, rho, lambd, t, r[i], K[i]))


In [ ]:
err

{'a': 0.01645735594931018,
 'b': 0.029056788511846728,
 'c': 0.01648968797869574,
 'd': 0.017711037300199548,
 'e': 0.030530149798396026}

In [ ]:
def calcImpl(x):
  v0, kappa, theta, sigma, rho, lambd = [param for param in x]
  P = heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
  impl = py_vollib_vectorized.vectorized_implied_volatility(P, S0, K, tau, r, flag = ['c'], q=0, model='black_scholes',return_as='numpy', on_error='ignore')
  return impl

In [ ]:
df_call['predIV']=calcImpl(params['d'].tolist())

<ipython-input-48-7b73f4590402>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  result[i], _ = quadrature(integrand, 0, 100, maxiter = 25, args=(S0, v0, kappa, theta, sigma, rho, lambd, t, r[i], K[i]))


In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs import Surface

fig = go.Figure(data=[go.Mesh3d(x=df_call['tau'], y=df_call['Strike'], z=df_call['IV']/100, color='mediumblue', opacity=0.55)])
fig.add_scatter3d(x=df_call['tau'], y=df_call['Strike'], z=df_call['predIV'], mode='markers')
fig.update_layout(
    title_text='Market Prices (Mesh) vs Calibrated Heston Prices (Markers)',
    scene = dict(xaxis_title='TIME (Years)',
                    yaxis_title='STRIKES (Pts)',
                    zaxis_title='INDEX OPTION PRICE (Pts)'),
    height=800,
    width=800
)
fig.show()

In [ ]:
def calibrate_heston(df, S0, x, r):
    
  K=df['Strike']
  tau=df['tau']
  iv = df['IV']/100
  r=df['r']
  S0=S0

  #market_ivol = heston_ivol(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r, iv)
  res = least_squares(heston_ivol, x0=np.array(x), verbose=0, max_nfev=100, method='trf',
                      bounds=([0, 1e-3, 1e-3, 1e-2, -1, -1], [np.inf, np.inf, np.inf, np.inf, 0, np.inf]))
  return res.x

In [ ]:
params_trf={}
for i in v0_dict.keys():
  try:
    params_trf[i]=calibrate_heston(df_call, S0, [v0_dict[i], kappa_dict[i], theta_dict[i], sigma_dict[i], rho_dict[i], lambd_dict[i]], r)
  except:
    params_trf[i]='not avaible'

In [ ]:
err_tfr = {}
for i in params_trf.keys():
  if params_trf[i]!='not avaible':

    err_tfr[i]=(heston_ivol(params_trf[i].tolist())**2).sum()/K.shape[0]

In [ ]:
params_trf

{'a': 'not avaible',
 'b': array([ 9.99999909e-02,  5.32944889e-02,  1.00005676e-03,  1.00000000e-02,
        -2.47716899e-15,  5.17471702e+00]),
 'c': 'not avaible',
 'd': 'not avaible',
 'e': array([ 9.99999994e-02,  4.10996007e+00,  1.00163606e-03,  1.00000689e-02,
        -7.45448013e-08,  1.16907789e+00])}

In [ ]:
err_tfr

{'b': 0.029765941777227474, 'e': 0.029769106607476094}

In [ ]:
df_call['predIV']=calcImpl(params_trf['b'].tolist())

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs import Surface

fig = go.Figure(data=[go.Mesh3d(x=df_call['tau'], y=df_call['Strike'], z=df_call['IV']/100, color='mediumblue', opacity=0.55)])
fig.add_scatter3d(x=df_call['tau'], y=df_call['Strike'], z=df_call['predIV'], mode='markers')
fig.update_layout(
    title_text='Market Prices (Mesh) vs Calibrated Heston Prices (Markers)',
    scene = dict(xaxis_title='TIME (Years)',
                    yaxis_title='STRIKES (Pts)',
                    zaxis_title='INDEX OPTION PRICE (Pts)'),
    height=800,
    width=800
)
fig.show()